# Training and deploying a tabular model using Vertex custom training job - Part 2

![Training pipeline](../images/custom-tabular.png)

In [1]:
import os
import pprint
import pandas as pd
import tensorflow as tf
import time

from google.cloud import aiplatform as vertex_ai
from google.cloud.aiplatform_v1beta1 import types

## Configure GCP settings


In [2]:
PREFIX = 'jkwst1'

shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
PROJECT = shell_output[0]
print("Project ID: ", PROJECT)

Project ID:  jk-wst1


In [3]:
REGION = 'us-central1'
STAGING_BUCKET = f'gs://{PREFIX}-bucket'

## Deploying a model to Vertex AI

### Initialize Vertex AI SDK

In [4]:
vertex_ai.init(
    project=PROJECT,
    location=REGION,
    staging_bucket=STAGING_BUCKET
)

### List custom jobs

In [5]:
search_filter = 'display_name="{}_CUSTOM_SCRIPT"'.format(PREFIX)

for job in vertex_ai.CustomJob.list(filter=search_filter, order_by='create_timedesc'):
    print(job.display_name, job.resource_name, job.state, job.create_time)

jkwst1_CUSTOM_SCRIPT projects/630263135640/locations/us-central1/customJobs/4758497209014550528 JobState.JOB_STATE_SUCCEEDED 2021-06-28 17:18:32.031881+00:00


### Wait for the oldest custom job to complete

In [6]:
while True:
    job_state = job.state
    if job_state != vertex_ai.gapic.JobState.JOB_STATE_SUCCEEDED:
        print("Job has not completed:", job_state)
        if (job_state == vertex_ai.gapic.JobState.JOB_STATE_FAILED or 
           job_state == vertex_ai.gapic.JobState.JOB_STATE_CANCELLED):
            break
    else:
        print("Job has completed")
        break
    time.sleep(60)

Job has completed


### Retrieve model artifacts

In [7]:
saved_model_path = '{}/model'.format(job.job_spec.base_output_directory.output_uri_prefix)

In [8]:
!gsutil ls {saved_model_path}

gs://jkwst1-bucket/jobs/jkwst1_CUSTOM_SCRIPT/20210628_171831/model/
gs://jkwst1-bucket/jobs/jkwst1_CUSTOM_SCRIPT/20210628_171831/model/saved_model.pb
gs://jkwst1-bucket/jobs/jkwst1_CUSTOM_SCRIPT/20210628_171831/model/assets/
gs://jkwst1-bucket/jobs/jkwst1_CUSTOM_SCRIPT/20210628_171831/model/variables/


### Inspect the model

In [9]:
!saved_model_cli show --dir {saved_model_path} --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['dropoff_grid'] tensor_info:
      dtype: DT_STRING
      shape: (-1, 1)
      name: serving_default_dropoff_grid:0
  inputs['euclidean'] tensor_info:
      dtype: DT_DOUBLE
      shape: (-1, 1)
      name: serving_default_euclidean:0
  inputs['payment_type'] tensor_info:
      dtype: DT_STRING
      shape: (-1, 1)
      name: serving_default_payment_type:0
  inputs['pickup_grid'] tensor_info:
      dtype: DT_STRING
      shape: (-1, 1)
      name: serving_default_pickup_grid:0
  inputs['trip_day'] tensor_info:
      dtype: DT_INT64
      shape: (-1, 1)
      name: serving_default_trip_day:0
  inputs['trip_day_of_week'] tensor_info:
      dtype: DT_INT64
      shape: (-1, 1)
      name: serving_default_trip_day_of_week:0
  inputs['trip_hour'] tensor_info:
      dtype: DT_INT64
      shape: (-1, 1)
      name: serving_default_trip_hour:0
  inputs['trip_miles'] tensor_info:
      dtype: DT_DOUBLE
      shape: (-1

### Upload the model

In [10]:
display_name = f'{PREFIX} Chicago taxi tip classifier'
description = 'Chicago taxi tip TensorFlow classifier'
serving_image_uri = 'us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-4:latest'

model = vertex_ai.Model.upload(
    display_name=display_name,
    description=description,
    artifact_uri=saved_model_path,
    serving_container_image_uri=serving_image_uri
)

model.wait()

INFO:google.cloud.aiplatform.models:Creating Model
INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/630263135640/locations/us-central1/models/194930217465151488/operations/3744640010943463424
INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/630263135640/locations/us-central1/models/194930217465151488
INFO:google.cloud.aiplatform.models:To use this Model in another session:
INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/630263135640/locations/us-central1/models/194930217465151488')


### Create an endpoint

In [11]:
display_name = f'{PREFIX} Taxi tip classifier endpoint'

endpoint = vertex_ai.Endpoint.create(display_name=display_name)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/630263135640/locations/us-central1/endpoints/2787499470923759616/operations/7647009088059998208
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/630263135640/locations/us-central1/endpoints/2787499470923759616
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/630263135640/locations/us-central1/endpoints/2787499470923759616')


### Deploy the Model

In [12]:
deployed_model_display_name = f'{PREFIX}-taxi-v1'
traffic_percentage = 100
machine_type = 'n1-standard-4'

endpoint = model.deploy(
        endpoint=endpoint,
        deployed_model_display_name=deployed_model_display_name,
        machine_type=machine_type,
        traffic_percentage=traffic_percentage)


INFO:google.cloud.aiplatform.models:Deploying model to Endpoint : projects/630263135640/locations/us-central1/endpoints/2787499470923759616
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/630263135640/locations/us-central1/endpoints/2787499470923759616/operations/5474022267853733888
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/630263135640/locations/us-central1/endpoints/2787499470923759616


## Invoking the deployed model using Vertex SDK

### Get the endpoint

In [13]:
filter = f'display_name="{PREFIX} Taxi tip classifier endpoint"'

for endpoint_info in vertex_ai.Endpoint.list(filter=filter):
    print(endpoint_info)
    
endpoint = vertex_ai.Endpoint(endpoint_info.resource_name)

resource name: projects/630263135640/locations/us-central1/endpoints/2787499470923759616


### Call the endpoint

In [14]:
test_instances = [  
    
    {
        "dropoff_grid": ["POINT(-87.6 41.9)"],
        "euclidean": [2064.2696],
        "payment_type": ["Credit Card"],
        "pickup_grid": ["POINT(-87.6 41.9)"],
        "trip_miles": [1.37],
        "trip_day": [12],
        "trip_hour": [16],
        "trip_month": [2],
        "trip_day_of_week": [4],
        "trip_seconds": [555]
    }
]

predictions = endpoint.predict(instances=test_instances)
prob = tf.nn.sigmoid(predictions[0])
print('Probability of tip > 20%:', prob.numpy())

Probability of tip > 20%: [[0.7745298]]


## Batch prediction

### Get the model

In [15]:
filter = f'display_name="{PREFIX} Chicago taxi tip classifier"'

for model_info in vertex_ai.Model.list(filter=filter):
    print(model_info)

resource name: projects/630263135640/locations/us-central1/models/194930217465151488


In [16]:
model = vertex_ai.Model(model_info.resource_name)

### Start a batch prediction job

In [23]:
gcs_path_to_instances = f'{STAGING_BUCKET}/batch_data/batch_instances.jsonl'
gcs_destination_prefix = f'{STAGING_BUCKET}/batch_predictions'

job_display_name = job_name = 'BATCH_PREDICT_{}'.format(time.strftime("%Y%m%d_%H%M%S"))

In [24]:
batch_prediction_job = model.batch_predict(
    job_display_name=job_display_name,
    instances_format='jsonl',
    gcs_source=gcs_path_to_instances,
    predictions_format='jsonl',
    machine_type='n1-standard-4',
    gcs_destination_prefix=gcs_destination_prefix
)

INFO:google.cloud.aiplatform.jobs:Creating BatchPredictionJob
INFO:google.cloud.aiplatform.jobs:BatchPredictionJob created. Resource name: projects/630263135640/locations/us-central1/batchPredictionJobs/2293902312936046592
INFO:google.cloud.aiplatform.jobs:To use this BatchPredictionJob in another session:
INFO:google.cloud.aiplatform.jobs:bpj = aiplatform.BatchPredictionJob('projects/630263135640/locations/us-central1/batchPredictionJobs/2293902312936046592')
INFO:google.cloud.aiplatform.jobs:View Batch Prediction Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/2293902312936046592?project=630263135640
INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/630263135640/locations/us-central1/batchPredictionJobs/2293902312936046592 current state:
JobState.JOB_STATE_RUNNING
INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/630263135640/locations/us-central1/batchPredictionJobs/2293902312936046592 current state:
JobState.JOB_STAT

RuntimeError: Job failed with:
code: 8
message: "Startup of the worker pool in zone us-central1-a failed to bring up any of the desired 1 workers. Try a different region, or try again later."


## Clean up

### Undeploy models

In [16]:
endpoint.list_models()

[id: "4940290461551034368"
model: "projects/910094146258/locations/us-central1/models/6871094452576845824"
display_name: "jk1-taxi-v1"
create_time {
  seconds: 1624406545
  nanos: 574565000
}
dedicated_resources {
  machine_spec {
    machine_type: "n1-standard-4"
  }
  min_replica_count: 1
  max_replica_count: 1
}
]

In [17]:
endpoint.undeploy_all()

INFO:google.cloud.aiplatform.models:Undeploying Endpoint model: projects/910094146258/locations/us-central1/endpoints/1568149871813197824
INFO:google.cloud.aiplatform.models:Undeploy Endpoint model backing LRO: projects/910094146258/locations/us-central1/endpoints/1568149871813197824/operations/5341388180195115008
INFO:google.cloud.aiplatform.models:Endpoint model undeployed. Resource name: projects/910094146258/locations/us-central1/endpoints/1568149871813197824


resource name: projects/910094146258/locations/us-central1/endpoints/1568149871813197824

### Delete endpoint

In [18]:
endpoint.delete()

INFO:google.cloud.aiplatform.base:Deleting Endpoint : projects/910094146258/locations/us-central1/endpoints/1568149871813197824
INFO:google.cloud.aiplatform.base:Delete Endpoint  backing LRO: projects/910094146258/locations/us-central1/operations/5665647353365790720
INFO:google.cloud.aiplatform.base:Endpoint deleted. . Resource name: projects/910094146258/locations/us-central1/endpoints/1568149871813197824


### Delete model

In [19]:
model.delete()

INFO:google.cloud.aiplatform.base:Deleting Model : projects/910094146258/locations/us-central1/models/6871094452576845824
INFO:google.cloud.aiplatform.base:Delete Model  backing LRO: projects/910094146258/locations/us-central1/operations/945663837648977920
INFO:google.cloud.aiplatform.base:Model deleted. . Resource name: projects/910094146258/locations/us-central1/models/6871094452576845824
